# CSCI 303
# Introduction to Data Science
<p/>
### 10 - Exploratory Data Analysis

![Exploratory data analysis](eda.png)

## This Lecture
---
- Explore the Boston Housing data set

The obligatory setup code...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import sklearn.datasets

from pandas import Series, DataFrame

plt.style.use('seaborn-whitegrid')

%matplotlib inline

## The Boston Housing Dataset
---
A well known and heavily studied dataset for statistical inference.

Available in the scikit-learn package, or many sources online.

In [3]:
raw = sk.datasets.load_boston()
boston = DataFrame(raw.data, columns=raw.feature_names)
boston['MEDV'] = raw.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Basic Statistics
---
pandas provides the `describe` function (similar to R's `summary`):

In [4]:
boston.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## What Shall We Explore?
---
Some ideas:

- distributions of individual inputs
- correlations between pairs of inputs and/or the target
- your suggestion here

## Distributions
---
Often best explored via histogram.

A histogram divides data into (usually) even sized *bins*, then counts the frequency of occurrence of samples in each bin.

For example, let's look at average number of rooms per dwelling.

In [ ]:
plt.hist(boston['RM'])
plt.show()

Very normal looking, isn't it?  We can vary the number of bins for more or less precision.

In [ ]:
plt.hist(boston['RM'], bins=20)
plt.show()

How about crime?

In [ ]:
plt.hist(boston['CRIM'], bins=20)
plt.show()

## Correlations
---
Often best explored via a scatter plot.

I theorize that there will be a correlation between percentage of industrial zoning and nitric oxide concentrations.  Let's take a look:

In [ ]:
plt.scatter(boston['INDUS'], boston['NOX'])
plt.xlabel('INDUS'); plt.ylabel('NOX');
plt.show()

There seems to be an odd artifact at around 18% on the INDUS axis.

Let's take a closer look at the INDUS data.

In [ ]:
plt.hist(boston['INDUS'], bins=range(25))
plt.show()

In [ ]:
boston['INDUS'].value_counts().head()

This spike at 18.10 seems suspicious.  Some kind of default?

In [ ]:
b2 = boston[boston['INDUS'] == 18.10]
b2.describe()

Four of the other columns have stddev = 0 when filtered on this value.

In [ ]:
b2

In [ ]:
356+132

What are the chances that 132 sequential entries have the same data for ZN, INDUS, RAD, TAX, and PTRATIO?

Let's set this aside for a moment and explore some other correlations.

We can drive plots directly from pandas, too, which provides some extra benefits - like axes labeling.

Let's look at # of rooms versus median value:

In [ ]:
boston.plot(kind='scatter', x='RM', y='MEDV')
plt.show()

Not too surprising, there seems to be a strong correlation between number of rooms and median value.

Now, though, we seem to have some other "suspicious" data - look at all those houses at the top!

In [ ]:
boston['MEDV'].plot(kind='hist', bins=15)

In [ ]:
boston['MEDV'].value_counts().iloc[:10]

In [ ]:
b3 = boston[boston['MEDV']==50]
b3

I'm quite suspicious that this value is some kind of data-entry default.

1. It's a round number
2. It's the maximum
3. It explains at least some big outliers: tracts where the average rooms per house < 5 AND the median value is 50,000 (and not with any obvious other great things going on)

For now, let's remove that data. It might not be justified, but without access to the original data collection info, it makes the most sense to me.

In [ ]:
bfix1 = boston[boston['MEDV'] != 50.0]
bfix1.plot(kind='scatter', x='RM', y='MEDV')

I'm curious about some of these other outliers.  I'm going to add in some other variables using color cues, just to see if they highlight the outliers.

In [ ]:
bfix1.plot(kind='scatter', x='RM', y='MEDV', c='CHAS', colormap='Accent')

That wasn't helpful.  What about our industrial zoning variable?

In [ ]:
bfix1.plot(kind='scatter', x='RM', y='MEDV', c='INDUS', colormap='Blues_r')

Hm.  I have a theory... not much of one, though.

In [ ]:
bfix2 = bfix1[bfix1['INDUS'] != 18.10]
bfix2.plot(kind='scatter', x='RM', y='MEDV', c='INDUS', colormap='Blues_r')

So this plot now actually makes sense; all the outliers vanished when we removed some suspicious data.

OTOH, we almost certainly lost some good data.

Was removing data the right thing to do?

In [ ]:
bfix2.shape, boston.shape

Other questions we could explore:
    
- what is the deal with houses on the Charles River?
- how do each of the remaining variables correlate with median value?
- how do DIS, RAD, and INDUS relate to each other?
- is there a relationship between crime and the age of the neighborhood?
- is PTRATIO relevant to anything?

In [ ]:
plt.hist(bfix2[bfix2['CHAS']==0]['MEDV'], bins=7)
plt.hist(bfix2[bfix2['CHAS']==1]['MEDV'], bins=7)
plt.show()

In [ ]:
plt.subplot(2,1,1)
plt.hist(bfix2[bfix2['CHAS']==0]['MEDV'], bins=range(5,50,5))
plt.subplot(2,1,2)
plt.hist(bfix2[bfix2['CHAS']==1]['MEDV'], bins=range(5,50,5), color='red')
plt.show()

In [ ]:
bfix2['CHAS'].value_counts()

In [ ]:
plt.subplot(2,1,1)
plt.hist(boston[boston['CHAS']==0]['MEDV'], bins=range(5,55,5))
plt.subplot(2,1,2)
plt.hist(boston[boston['CHAS']==1]['MEDV'], bins=range(5,55,5), color='red')
plt.show()

In [ ]:
for f in raw.feature_names:
    plt.subplot(1,2,1)
    plt.hist(bfix2[f])
    plt.xlabel(f)
    plt.subplot(1,2,2)
    plt.scatter(bfix2[f], bfix2['MEDV'])
    plt.xlabel(f)
    plt.ylabel('MEDV')
    plt.show()
    

In [ ]:
bfix2.plot(kind='scatter', x='INDUS', y='DIS', c='RAD', colormap='Blues')

In [ ]:
bfix2.plot(kind='scatter', x='AGE', y='CRIM')

In [ ]:
inc = 10
x = range(10,101,inc)
y = []
for i in x:
    bset = bfix2[bfix2['AGE'] <= i]
    bset = bset[bset['AGE'] > (i - inc)]
    y.append(bset['CRIM'].std())

plt.plot(x, y, '-s')

In [ ]:
bfix3 = boston[boston['INDUS'] != 18.10]

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [ ]:
from sklearn.model_selection import train_test_split
bfix3_train, bfix3_test = train_test_split(bfix3)

In [ ]:
lr.fit(bfix3_train[raw.feature_names], bfix3_train['MEDV'])

In [ ]:
lr.score(bfix3_test[raw.feature_names], bfix3_test['MEDV'])

In [ ]:
lr2 = LinearRegression()
boston_train, boston_test = train_test_split(boston)
lr2.fit(boston_train[raw.feature_names], boston_train['MEDV'])
lr2.score(boston_test[raw.feature_names], boston_test['MEDV'])

In [ ]:
lr2.score(bfix3_test[raw.feature_names], bfix3_test['MEDV'])

In [ ]:
#If we want to train on just a few features, just filter that one
boston = DataFrame(raw.data, columns = raw.feature_names)
boston['MEDV'] = raw.target

#create a lr object
lrOneFeat = LinearRegression()

#split the data between training and test sets
boston_train, boston_test = train_test_split(boston)

#train the lr object
#lrOneFeat.fit(boston_train[raw.feature_names[0:2]], boston_train['MEDV'])
#lrOneFeat.fit(boston_train[['CRIM']], boston_train['MEDV'])
lrOneFeat.fit(boston_train[['CRIM','RM']], boston_train['MEDV'])

#test the trained model, print the score
#lrOneFeat.score(boston_test[raw.feature_names[0:2]], boston_test['MEDV'])
#lrOneFeat.score(boston_test[['CRIM']], boston_test['MEDV'])
lrOneFeat.score(boston_test[['CRIM','RM']], boston_test['MEDV'])

In [ ]:
print(raw.DESCR)